# Hosting FLAIR models on Sagemaker

Below is some initial imports and configuration.

In [2]:
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role
import json
import pickle
import flair


role = get_execution_role()

In [3]:
import sagemaker
from time import gmtime, strftime

sess = sagemaker.Session() # can use LocalSession() to run container locally

bucket = sess.default_bucket()
region = "us-east-2"
account = sess.boto_session.client('sts').get_caller_identity()['Account']
prefix_input = 'flair-input'
prefix_output = 'flair-ouput'

## Define parameters of your container

In this notebook, we are using PyTorch container provided by Sagemaker. Full list of available deep learning containers is here: https://docs.aws.amazon.com/deep-learning-containers/latest/devguide/deep-learning-containers-images.html

If needed, Sagemaker containers can be extended.  Typically, this is needed when you need to have a specific custom setup or when requirement.txt doesn't allow you to build/install desired version of Python packages.

In [4]:
# Using Sagemaker PyTorch 1.5 serving container

container_serving = "pytorch-inference" # your container name
tag = "1.5.0-gpu-py36-cu101-ubuntu16.04" # you can have several version of container available
image = '763104351884.dkr.ecr.{}.amazonaws.com/{}:{}'.format(region, container_serving, tag)

print("Following container will be used for hosting: ",image)

Following container will be used for hosting:  763104351884.dkr.ecr.us-east-2.amazonaws.com/pytorch-inference:1.5.0-gpu-py36-cu101-ubuntu16.04


## Deploy Realtime endpoint

Sagemaker Realtime endpoints allows you to get inference in real-time by sending HTTP requests. In this example, we'll deploy a new endpoint. Inference code is provided in `sequence_tagger_serving.py` file, `requirements.txt` specify which additional Python packages you need to install additionally in Sagemaker PyTorch container.

In [5]:
from sagemaker.pytorch import PyTorchModel, PyTorch, PyTorchPredictor
from sagemaker.estimator import Estimator, Model

remote_model = PyTorchModel(name = "flair-tagger-v3",
                            model_data="s3://vadimd-empty-bucket/placeholder/emptyfile.tar.gz", # This will be replaced with location of trained model artifacts. 
                                                                                                # For now, default pre-trained FLAIR model will be used.
                            role=role,
                            sagemaker_session = sess,
                            entry_point="sequence_tagger_serving.py",
                            source_dir="serving_sources",
                            framework_version="1.5", py_version="3.6",
                            image=image)

In [6]:
remote_predictor = remote_model.deploy(instance_type='ml.g4dn.xlarge',
                                       initial_instance_count=1,
                                       update_endpoint = True, # comment or False if endpoint doesns't exist
                                       endpoint_name="sequence-endpoint-v1", # define a unqie endpoint name; if ommited, Sagemaker will generate it based on used container
                                       wait=False
                                      )

In [25]:
# sentence_to_predict = "Berlin is the capital and largest city of Germany by both area and population."
sentence_to_predict  = "New York City (NYC), often called The City or simply New York (NY), is the most populous city in the United States."

client = boto3.client('sagemaker-runtime')
content_type = 'application/json'
accept_type = "pickle"
headers = {'content-type': content_type}
payload = json.dumps(sentence_to_predict)
endpoint_name = "sequence-endpoint-v1"

response = client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=payload,
    ContentType=content_type,
    Accept = accept_type
)

prediction_obj = pickle.loads(response['Body'].read())
print(prediction_obj)

[Sentence: "New York City (NYC), often called The City or simply New York (NY), is the most populous city in the United States."   [− Tokens: 22  − Token-Labels: "New <B-LOC> York <I-LOC> City <E-LOC> (NYC), often called The <B-LOC> City <E-LOC> or simply New <B-LOC> York <E-LOC> (NY), is the most populous city in the United <B-LOC> States. <E-LOC>"]]


# Use Batch Transform inference

Sagemaker also allows to run inference in a batch format for multiple inputs. In this case, your input data is stored in S3 object, the output file with batch predictions will be also available as object on S3. 

We'll use the same container image as for realtime inference. The only difference, that this time both request content type and response content type will be "text/csv".


In [8]:
bucket = "vadimd-batch-transform"

input_data_path = 's3://{}/{}/{}'.format(bucket, 'flair', 'sample.csv')
output_data_path = 's3://{}/{}'.format(bucket, 'flair-output')

transform_job = sagemaker.transformer.Transformer(
    model_name = "flair-tagger-v3",
    instance_count = 1,
    instance_type = 'ml.p2.xlarge',
    strategy = 'SingleRecord',
    assemble_with = 'Line',
    output_path = output_data_path,
#     base_transform_job_name='inference-pipelines-batch',
    sagemaker_session=sess,
    accept = "text/csv")

transform_job.transform(data = input_data_path, 
                        content_type = "text/csv", 
                        split_type = 'Line')

In [ ]:
After batch transform job successfully 